In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
import numpy as np
import matplotlib.pyplot as plt
import Markov_Models as mm
from scipy import stats
from scipy import interpolate
from scipy import spatial
from scipy import integrate
import os
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error

path="/home/ebeyerle/Desktop/PCA/UBQN/400ns/"
savepath='/home/ebeyerle/Desktop/params/lipshitz/pbcs/'
os.chdir(path)

In [2]:
def ZTS(nframes,nstep,first_mode,last_mode):   
    #Do a little ZTS to get a good starting configuration
    bins=50
    xx,yy=np.meshgrid(np.linspace(0,180,bins),np.linspace(0,360,bins))
    #mode=1
    nfrs=1
    #nframes=50
    barlist=[]
    pointslist=[]
    for mode in range(first_mode,last_mode):
        with open(path+'anly_'+str(mode)+'.dat','r') as data:
            x=[]
            y=[]
            for line in data:
                nfrs=nfrs+1
                p=line.split()
                x.append(float(p[0]))
                y.append(float(p[1]))
        print(nfrs)
        data=[np.column_stack([x,y])]
        model = mm.MSM(data)
        his0 = model.histogram(0, bins=50)
        his1 = model.histogram(1, bins=50)
        his = model.histogram(bins=50)
        ext=model.extent
        lst=np.zeros(bins*bins)
        chk=his.T/his.sum()
        counter=-1
        for i in range(len(chk[0,:])):
            for j in range(len(chk[:,0])):
                counter=counter+1
                lst[counter]=chk[i][j]
        counter=0
        fes=np.zeros((bins,bins))
        for i in range(len(chk[0,:])):
            for j in range(len(chk[:,0])):
                if (lst[counter] <= 0):
                    fes[i][j]=-np.log(1/nfrs)
                    counter=counter+1
                else:
                    fes[i][j]=-np.log(lst[counter])
                    counter=counter+1
        min1=np.min(fes)
        for i in range(bins):
            for j in range(bins):
                fes[i][j]=fes[i][j]-min1
        ##plt.contourf(xx,yy,fes, 25, cmap='gnuplot')
        ##plt.show()
        theta=np.linspace(0,180,bins)
        phi=np.linspace(0,360,bins)
        #Smooth the surface with k-nearest neighbors
        n_values = 100
        alpha = 2

        x0, x1 = np.meshgrid(theta,phi)
        X = np.column_stack([x0.ravel(), x1.ravel()])
        score = []
        for n_neighbor in np.arange(alpha, n_values * alpha + 1, alpha):
            for train_index, test_index in ShuffleSplit(n_splits=1, test_size=0.40, random_state=24).split(X):
                X_train, y_train = X[train_index], fes.ravel()[train_index]
                X_test, y_test = X[test_index], fes.ravel()[test_index]

            knn = KNeighborsRegressor(n_neighbors=n_neighbor).fit(X_train, y_train)
            score.append(mean_squared_error(knn.predict(X_test), y_test))
        if np.argmin(score) == 0:
            n_neighbors=2
            print("Hmm... no nearest neighbors. Setting k:=1.")
        else:
            n_neighbors = np.argmin(score) * alpha        
        print("k = {:d}".format(np.argmin(score)))
        knn = KNeighborsRegressor(n_neighbors=n_neighbors).fit(X, fes.ravel())
        fes_knn = knn.predict(X).reshape(bins, bins)
        dVy,dVx=np.gradient(fes_knn)
        dVx=interpolate.interp2d(theta,phi,dVx,kind='quintic')
        dVy=interpolate.interp2d(theta,phi,dVy,kind='quintic')
        #plt.close()
        #plt.contourf(xx,yy,fes,cmap='gnuplot')
        #plt.quiver(xx,yy,-dVx(theta,phi),-dVy(theta,phi))
        ##plt.show()
        #Now convert the code from the ZTS Matlab code
        tol1=0.00001 #Convergence criterion
        tollist=[]
        flag1=1
        nstepplot=100
        #step=10000
        g1=np.linspace(0,1,nframes)
        #Perform the interpolation; same as above
        min1=np.where(fes == 0.0)
        ya=min1[0][0]
        xa=min1[1][0]
        xb=140
        yb=300
        x=(xb-xa)*g1+xa
        y=(x-xa)*(yb-ya)/(xb-xa)+ya
        dx=x-np.roll(x,1,axis=0)
        dy=y-np.roll(y,1,axis=0)
        dx[0]=0
        dy[0]=0
        lxy=np.cumsum(np.sqrt(dx**2+dy**2))
        lxy=lxy/lxy[nframes-1]
        #Honestly not sure why these two lines are here; the x, y that are returned
        #are identical to the x, y generated by the linear interpolation.
        x=interpolate.interp1d(lxy,x)(g1)
        y=interpolate.interp1d(lxy,y)(g1)
        xi=x
        yi=y
        ##plt.plot(xi,yi,'w',lw=1)
        #plt.scatter(xi,yi,marker='x',c='w')
        #plt.plot(x,y,'b')
        counter=1
        ##plt.savefig("/home/ebeyerle/Desktop/fifty_fes_animate_mode"+str(mode)+"frame00"+str(counter)+".jpeg")
        #plt.close()
        ##plt.show()
        laplist=[]
        grady,gradx=np.gradient(fes_knn)
        lapxy,lapxx=np.gradient(gradx)
        lapyy,lapyx=np.gradient(grady)
        h=1/(np.array([lapxx.max(),lapxy.max(),lapyx.max(),lapyy.max()]).max())
        #h=0.1#Just a parameter -- proabably need to adjust to get good convergence
        print('h=',h)
        for i in range(nstep):
            dVy,dVx=np.gradient(fes_knn)
            dVx=interpolate.interp2d(theta,phi,dVx,kind='quintic')
            dVy=interpolate.interp2d(theta,phi,dVy,kind='quintic')
            x0=x
            y0=y
            # Evolve the string using steepest descent
            for j in range(len(x)):
                if y[j] <=0.0: 
                    x[j]=x[j]-h*dVx(x[j],360.0+y[j])[0]
                    y[j]=y[j]-h*dVy(x[j],360.0+y[j])[0]

                if y[j] >=360.0:
                    x[j]=x[j]-h*dVx(x[j],y[j]-360.0)[0]
                    y[j]=y[j]-h*dVy(x[j],y[j]-360.0)[0]

                if x[j] <=0.0:
                    x[j] = 0.0
                    x[j]=x[j]-h*dVx(x[j],y[j])[0]
                    y[j]=y[j]-h*dVy(x[j],y[j])[0]

                if x[j] >= 180.0:
                    x[j]=180.0
                    x[j]=x[j]-h*dVx(x[j],y[j])[0]
                    y[j]=y[j]-h*dVy(x[j],y[j])[0]

                else:
                    x[j]=x[j]-h*dVx(x[j],y[j])[0]
                    y[j]=y[j]-h*dVy(x[j],y[j])[0]
                #Enforce PBCs
                #KLUDGE: Not letting them cross for now
            #Reparameterize
            dx=x-np.roll(x,1,axis=0)
            dy=y-np.roll(y,1,axis=0)
            dx[0]=0
            dy[0]=0
            #Find the string length...
            lxy=np.cumsum(np.sqrt(dx**2+dy**2))
            lxy=lxy/lxy[nframes-1]
            #...and place each state at an equally-spaced length along the string
            x=interpolate.interp1d(lxy,x)(g1)
            y=interpolate.interp1d(lxy,y)(g1)
            tol=(np.linalg.norm(x-x0)+np.linalg.norm(y-y0))/(nframes)
            tollist.append(tol)
            #if len(tollist) >=3:
            #    if tollist[i-3]-tollist[i] == 0:
            #        print('converged')
            #        break
            if tol <= tol1: #Has the string converged?
                break #It has converged!
            if i%nstepplot == 0:
                print(tol)
        print('\n')
        print('\n')
        print('ZTS calculation with ',nframes,' images.\n')
        if tol >= tol1:
            print('The method did not converge in ',nstep,' steps.')
        else:
            print('The method converged after ',nstep,' steps.')
        #Apply boundary conditions
        for i in range(len(x)):
            if y[i] >= 360.0:
                y[i]=y[i]-360.0
            elif y[i] <= 0.0:
                y[i]=y[i]+360.0
        #plt.plot(x,y)
        plt.contourf(xx,yy,fes,cmap='gnuplot')
        plt.quiver(xx,yy,-dVx(theta,phi),-dVy(theta,phi))
        plt.scatter(x,y,marker='o',c='w')
        #plt.plot(x,y,'b')
        plt.xlim((0,180))
        plt.ylim((0,360))
        plt.savefig(savepath+'ZTS-final-'+str(mode)+'.pdf')
        plt.close()
        ifes=interpolate.interp2d(theta,phi,fes,kind='quintic')
        points=[]
        for i in range(len(x)):
            points.append(ifes(x[i],y[i])[0])
            #print(points[i])
        pointslist.append(points)
        np.savetxt("ZTS-points"+str(mode),points)
        #np.savetxt("coords_"+str(mode)+".dat",np.column_stack([x,y]))
        #plt.plot(points,'k')
        #plt.xlabel('Image Number')
        #plt.ylabel('free energy ($k_BT$)')
        bar=np.max(np.array(points))-np.min(np.array(points))
        print('Barrier Height ($k_BT$): ',bar)
        barlist.append(bar)
        np.savetxt(savepath+"ZTS-barlist.dat",np.array(barlist))
        np.savetxt(savepath+"ZTS-coords_"+str(mode)+".dat",np.column_stack([x,y]))
    return x,y,fes

def FTS(nframes,nstep,n_strings,first_mode,last_mode,x,y,fes): 
    '''Args
    FTS(nframes,nstep,n_strings,first_mode,last_mode,x,y,fes)
    nframes: Desired number of images in the string
    nstep: Number of Brownian dynamics steps to take on the surface
    n_strings: Number of strings to simulate
    first_mode: First mode to sample
    last_mode: Last mode to sample
    x: Abcissa of the initial string
    y: Ordinate of the initial string
    fes: Free-energy surface on which to evolve the string
    '''
    
    bins=50
    xx,yy=np.meshgrid(np.linspace(0,180,bins),np.linspace(0,360,bins))
    #mode=1
    nfrs=1
    #nframes=50
    barlist=[]
    pointslist=[]
    theta=np.linspace(0,180,bins)
    phi=np.linspace(0,360,bins)
    kT=0.5
    for mode in range(first_mode,last_mode):
        #fig=#plt.figure(figsize=(12,8))
        dVy,dVx=np.gradient(fes)
        dVx=interpolate.interp2d(theta,phi,dVx,kind='quintic')
        dVy=interpolate.interp2d(theta,phi,dVy,kind='quintic')
        #plt.close()
        #plt.contourf(xx,yy,fes,cmap='gnuplot')
        #plt.xlabel(r'$\theta (deg)$')
        #plt.ylabel(r'$\phi (deg)$')
        #plt.quiver(xx,yy,-dVx(theta,phi),-dVy(theta,phi))
        ##plt.show()
        #Now convert the code from the ZTS Matlab code
        tol1=0.00001 #Convergence criterion
        tollist=[]
        flag1=1
        nstepplot=1000
        #nstep=10000
        g1=np.linspace(0,1,nframes)
        #Perform the interpolation; same as above
        ya=y[0]
        xa=x[0]
        xb=x[-1]
        yb=y[-1]
        x=(xb-xa)*g1+xa
        y=(x-xa)*(yb-ya)/(xb-xa)+ya
        dx=x-np.roll(x,1,axis=0)
        dy=y-np.roll(y,1,axis=0)
        dx[0]=0
        dy[0]=0
        lxy=np.cumsum(np.sqrt(dx**2+dy**2))
        lxy=lxy/lxy[nframes-1]
        #Honestly not sure why these two lines are here; the x, y that are returned
        #are identical to the x, y generated by the linear interpolation.
        x=interpolate.interp1d(lxy,x)(g1)
        y=interpolate.interp1d(lxy,y)(g1)
        xi=x
        yi=y
        ##plt.plot(xi,yi,'w',lw=1)
        #plt.scatter(xi,yi,marker='x',c='w')
        #plt.plot(x,y,'b')
        counter=1
        ##plt.savefig("/home/ebeyerle/Desktop/fifty_fes_animate_mode"+str(mode)+"frame00"+str(counter)+".jpeg")
        #plt.close()
        ##plt.show()
        laplist=[]
        grady,gradx=np.gradient(fes)
        lapxy,lapxx=np.gradient(gradx)
        lapyy,lapyx=np.gradient(grady)
        h=1/(np.array([lapxx.max(),lapxy.max(),lapyx.max(),lapyy.max()]).max())
        #n_strings=10
        #h=0.1#Just a parameter -- proabably need to adjust to get good convergence
        #Create an 'ensemble' of strings
        xstrings=x+np.random.normal(scale=np.sqrt(2*kT),size=(n_strings,nframes))
        ystrings=y+np.random.normal(scale=np.sqrt(2*kT),size=(n_strings,nframes))
        x0=np.zeros((n_strings,nframes))
        y0=np.zeros((n_strings,nframes))
        print('h=',h)
        for k in range(nstep):
            etax=np.random.randn(n_strings,len(x))
            etay=np.random.randn(n_strings,len(x))
            dVy,dVx=np.gradient(fes)
            dVx=interpolate.interp2d(theta,phi,dVx,kind='quintic')
            dVy=interpolate.interp2d(theta,phi,dVy,kind='quintic')
            for i in range(n_strings):
                x0[i]=xstrings[i]
                y0[i]=ystrings[i]
            # Evolve the string using steepest descent
            for j in range(len(x)):
                for i in range(n_strings):
                    if ystrings[i][j] <=0.0: 
                        xstrings[i][j]=xstrings[i][j]-h*dVx(xstrings[i][j],360+ystrings[i][j])[0]+h*np.sqrt(2*kT)*etax[i][j]
                        ystrings[i][j]=ystrings[i][j]-h*dVy(xstrings[i][j],360+ystrings[i][j])[0]+h*np.sqrt(2*kT)*etay[i][j]

                    if ystrings[i][j] >=360.0:
                        xstrings[i][j]=xstrings[i][j]-h*dVx(xstrings[i][j],ystrings[i][j]-360)[0]+h*np.sqrt(2*kT)*etax[i][j]
                        ystrings[i][j]=ystrings[i][j]-h*dVy(xstrings[i][j],ystrings[i][j]-360)[0]+h*np.sqrt(2*kT)*etay[i][j]

                    if xstrings[i][j] <=0.0:
                        xstrings[i][j] = 0.0
                        xstrings[i][j]=xstrings[i][j]-h*dVx(xstrings[i][j],ystrings[i][j])[0]+h*np.sqrt(2*kT)*etax[i][j]
                        ystrings[i][j]=ystrings[i][j]-h*dVy(xstrings[i][j],ystrings[i][j])[0]+h*np.sqrt(2*kT)*etay[i][j]

                    if xstrings[i][j] >= 180.0:
                        xstrings[i][j]=180.0
                        xstrings[i][j]=xstrings[i][j]-h*dVx(xstrings[i][j],ystrings[i][j])[0]+h*np.sqrt(2*kT)*etax[i][j]
                        ystrings[i][j]=ystrings[i][j]-h*dVy(xstrings[i][j],ystrings[i][j])[0]+h*np.sqrt(2*kT)*etay[i][j]

                    else:
                        xstrings[i][j]=xstrings[i][j]-h*dVx(xstrings[i][j],ystrings[i][j])[0]+h*np.sqrt(2*kT)*etax[i][j]
                        ystrings[i][j]=ystrings[i][j]-h*dVy(xstrings[i][j],ystrings[i][j])[0]+h*np.sqrt(2*kT)*etay[i][j]
                    #Enforce PBCs
            #Reparameterize
            for i in range(n_strings):
                dx=xstrings[i]-np.roll(xstrings[i],1,axis=0)
                dy=ystrings[i]-np.roll(ystrings[i],1,axis=0)
                dx[0]=0
                dy[0]=0
                #Find the string length...
                lxy=np.cumsum(np.sqrt(dx**2+dy**2))
                lxy=lxy/lxy[nframes-1]
                #...and place each state at an equally-spaced length along the string
                xstrings[i]=interpolate.interp1d(lxy,xstrings[i])(g1)
                ystrings[i]=interpolate.interp1d(lxy,ystrings[i])(g1)
            #if flag1 == 1 and i%nstepplot == 0:
            #    counter=counter+1
                ##plt.plot(x,y)
            #    if counter < 10:
            #        #plt.contourf(xx,yy,fes,cmap='gnuplot')
            #        #plt.xlabel(r'$\theta (deg)$')
            #        #plt.ylabel(r'$\phi (deg)$')
            #        #plt.quiver(xx,yy,-dVx(theta,phi),-dVy(theta,phi))
            #        #plt.scatter(x,y,marker='o',c='w')
            #        #plt.plot(x,y,'b')
                    ##plt.savefig("/home/ebeyerle/Desktop/FTS_animate_mode"+str(mode)+"frame00"+str(counter)+".jpeg")
            #        #plt.close()
            #    elif counter >= 10 and counter <100:
            #        #plt.contourf(xx,yy,fes,cmap='gnuplot')
            #        #plt.xlabel(r'$\theta (deg)$')
            #        #plt.ylabel(r'$\phi (deg)$')
            #        #plt.quiver(xx,yy,-dVx(theta,phi),-dVy(theta,phi))
            #        #plt.scatter(x,y,marker='o',c='w')
            #        #plt.plot(x,y,'b')
                    ##plt.savefig("/home/ebeyerle/Desktop/FTS_animate_mode"+str(mode)+"frame0"+str(counter)+".jpeg")
            #        #plt.close()
            #    else:
            #        #plt.contourf(xx,yy,fes,cmap='gnuplot')
            #        #plt.xlabel(r'$\theta (deg)$')
            #        #plt.ylabel(r'$\phi (deg)$')
            #        #plt.quiver(xx,yy,-dVx(theta,phi),-dVy(theta,phi))
            #        #plt.scatter(x,y,marker='o',c='w')
            #        #plt.plot(x,y,'b')
                    ##plt.savefig("/home/ebeyerle/Desktop/FTS_animate_mode"+str(mode)+"frame"+str(counter)+".jpeg")
            #        #plt.close()
            #    #plt.show()
            #Calculate the tolerance for one string
            tol=(np.linalg.norm(xstrings[0]-x0)+np.linalg.norm(ystrings[0]-y0))/(nframes)
            tollist.append(tol)
            #if len(tollist) >=3:
            #    if tollist[i-3]-tollist[i] == 0:
            #        print('converged')
            #        break
            if tol <= tol1: #Has the string converged?
                break #It has converged!
            if k%nstepplot == 0:
                print(tol)
        print('\n')
        print('\n')
        print('FTS calculation with ',nframes,' images.\n')
        if tol >= tol1:
            print('The method did not converge in ',nstep,' steps.')
        else:
            print('The method converged after ',nstep,' steps.')
        #Find the average string
        sumx=np.zeros(len(x))
        sumy=np.zeros(len(y))
        for j in range(len(x)):
            for i in range(n_strings):
                sumx[j]=sumx[j]+xstrings[i][j]
                sumy[j]=sumy[j]+ystrings[i][j]
            sumx[j]=sumx[j]/n_strings
            sumy[j]=sumy[j]/n_strings
            #avgx.append(sumx[j])
            #avgy.append(sumy[j])
        x=sumx#np.array(avgx)
        y=sumy#np.array(avgy)
        plt.contourf(xx,yy,fes,cmap='gnuplot')
        plt.xlabel(r'$\theta (deg)$')
        plt.ylabel(r'$\phi (deg)$')
        plt.quiver(xx,yy,-dVx(theta,phi),-dVy(theta,phi))
        plt.scatter(x,y,marker='o',c='w')
        plt.xlim((0,180))
        plt.ylim((0,360))
        plt.savefig(savepath+'FTS-final-'+str(mode)+'.pdf')
        plt.close()
        tx=np.roll(x,-1,axis=0)-np.roll(x,1,axis=0)
        ty=np.roll(y,-1,axis=0)-np.roll(y,1,axis=0)
        #Use trapezoidal integration
        from scipy import integrate
        Vz=integrate.cumtrapz(tx*dVx(x,y)+ty*dVy(x,y))
        Vz=0.5*Vz
        Vz=Vz-np.min(Vz)
        ntxy=np.sqrt(tx*tx+ty*ty)
        tx=tx/ntxy
        ty=ty/ntxy
        err=np.trapz(1-(tx*dVx(x,y)+ty*dVy(x,y))**2/(dVx(x,y)*dVx(x,y)+dVy(x,y)*dVy(x,y))/nframes)
        print('\n')

        #Plot the 1D FES along the string
        ifes=interpolate.interp2d(theta,phi,fes,kind='quintic')
        points=[]
        for i in range(len(x)):
            points.append(ifes(x[i],y[i])[0])
            #print(points[i])
        pointslist.append(points)

        #Save some imprtant stuff
        np.savetxt("FTS-points"+str(mode),points)
        #np.savetxt("coords_"+str(mode)+".dat",np.column_stack([x,y]))
        #plt.plot(points,'k')
        #plt.xlabel('Image Number')
        #plt.ylabel('free energy ($k_BT$)')
        bar=np.max(np.array(points))-np.min(np.array(points))
        print('Barrier Height ($k_BT$): ',bar)
        barlist.append(bar)
        np.savetxt("FTS-barlist.dat",np.array(barlist))
        np.savetxt("FTS-coords_"+str(mode)+".dat",np.column_stack([x,y]))
        np.save(savepath+'xstrings'+str(mode)+'.npy',xstrings)
        np.save(savepath+'ystrings'+str(mode)+'.npy',ystrings)
        np.save(savepath+'fes'+str(mode)+'.npy',fes)

        #Plot the transition tube and average string
        tube=np.zeros((4,nframes))
        for i in range(nframes):
            #Initialize the min and max for x, y
            minx=xstrings[0][i]
            miny=ystrings[0][i]
            maxx=xstrings[0][i]
            maxy=ystrings[0][i]
            maxj=0
            minj=0
            for j in range(n_strings):
                val=xstrings[j][i]
                if val <= minx:
                    minx=val
                    minj=j
                    #print(minx,minj)
                if val >= maxx:
                    maxx=val
                    maxj=j
                    #print(maxx,maxj)
            #print(minx,minj,maxx,maxj)
            miny=ystrings[minj][i]
            maxy=ystrings[maxj][i]
            tube[0][i]=minx
            tube[1][i]=miny
            tube[2][i]=maxx
            tube[3][i]=maxy
        plt.contourf(xx,yy,fes,cmap='gnuplot')
        plt.xlabel(r'$\theta\ (deg)$')
        plt.ylabel(r'$\phi\ (deg)$')
        plt.quiver(xx,yy,-dVx(theta,phi),-dVy(theta,phi))
        for i in range(len(xstrings)):
            plt.scatter(xstrings[i],ystrings[i],marker='o')
        plt.plot(tube[0],tube[1],lw=5,color='w')
        plt.plot(tube[2],tube[3],lw=5,color='w')
        plt.plot(x,y,color='r',lw=3)
        plt.xlim((0,180))
        plt.ylim((0,360))
        plt.save(savepath+'tube'+str(mode)+'.npy')
        plt.savefig(savepath+'FTS-tube-'+str(mode)+'.pdf')
    return x,y,xstrings,ystrings,fes

In [3]:
#Run it for all the modes
for i in range(1,229):
    x,y,fes=ZTS(50,10000,i,i+1)  
    x,y,fes=FTS(50,10000,i,i+1,x,y,fes) #Low precision 
    x,y,xstrings,ystrings,fes=FTS(200,10,1000,i,i+1,x,y,fes) #High precision 

1785715
k = 2
h= 0.597158696314
0.0345464233818
0.032561486094
0.0561546661494
0.0478772615979
0.0399805440986
0.046110754425
0.0308203386932
0.0314268495888
0.0315239079175
0.0317145000064
0.0320235877147
0.033585682433
0.0383010275167
0.0327960480606
0.0324764496473
0.0328527473656
0.0441818171786
0.0314232115481
0.0314140703701
0.0314140694428
0.0314140694295
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.0314140694294
0.031414069

TypeError: FTS() missing 1 required positional argument: 'fes'